In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

def deutsch_jozsa_n2(oracle_type='balanced'):
    """
    Implements Deutsch-Jozsa Algorithm for n=2 (3 qubits total).
    """
    # Initialize a quantum circuit with 3 qubits and 2 classical bits
    # Qubits 0, 1: Input Register
    # Qubit 2: Ancilla
    qc = QuantumCircuit(3, 2)
    
    # --- Step 0: Initialization ---
    # Inputs (q0, q1) are already |0>. 
    # Ancilla (q2) needs to be |1> for phase kickback.
    qc.x(2) 
    qc.barrier()
    
    # --- Step 1: Apply Hadamard Gates ---
    # Apply H to ALL qubits (inputs + ancilla)
    qc.h(0)
    qc.h(1)
    qc.h(2)
    qc.barrier()
    
    # --- Step 2: Apply the Oracle U_f ---
    if oracle_type == 'balanced':
        # Balanced Oracle Example: f(x0, x1) = x0 XOR x1
        # If x0 != x1, output 1. If x0 == x1, output 0.
        # We implement this using CNOTs from inputs to the ancilla.
        qc.cx(0, 2) 
        qc.cx(1, 2)
    elif oracle_type == 'constant':
        # Constant Oracle: f(x) = 0 
        # (Identity, do nothing)
        pass 
        
    qc.barrier()
    
    # --- Step 3: Apply Final Hadamard ---
    # Apply H to the Input Register only (q0, q1)
    qc.h(0)
    qc.h(1)
    qc.barrier()
    
    # --- Step 4: Measurement ---
    # Measure input qubits (0 and 1) into classical bits (0 and 1)
    qc.measure([0, 1], [0, 1])
    
    return qc

# Create the circuit
qc = deutsch_jozsa_n2(oracle_type='balanced')

# Draw the circuit diagram
print("Circuit Diagram:")
print(qc.draw(output='text'))

# --- Simulation ---
simulator = Aer.get_backend('aer_simulator')

# Transpile and run
transpiled_qc = transpile(qc, simulator)
result = simulator.run(transpiled_qc).result()
counts = result.get_counts()

print("\nMeasurement Result:")
print(f"Counts: {counts}")
print("-" * 30)
if '00' in counts and len(counts) == 1:
    print("Result: 00 -> Function is CONSTANT")
else:
    print("Result: NOT 00 -> Function is BALANCED")